# Background

This dataset contains the location, address, type, sale price, and sale date of building units sold. A reference on the trickier fields:
* BOROUGH : A digit code for the borough the property is located in; in order these are Manhattan (1), Bronx (2), Brooklyn (3), Queens (4), and Staten Island (5).
* BLOCK; LOT :The combination of borough, block, and lot forms a unique key for property in New York City. Commonly called a BBL.
* BUILDING CLASS AT PRESENT and BUILDING CLASS AT TIME OF SALE: : The type of building at various points in time.


Note that because this is a financial transaction dataset, there are some points that need to be kept in mind:
* Many sales occur with a nonsensically small dollar amount: $0 most commonly. These sales are actually transfers of deeds between parties: for example, parents transferring ownership to their home to a child after moving out for retirement.
* This dataset uses the financial definition of a building/building unit, for tax purposes. In case a single entity owns the building in question, a sale covers the value of the entire building. In case a building is owned piecemeal by its residents (a condominium), a sale refers to a single apartment (or group of apartments) owned by some individual.link dataset:[di sini](https://www.kaggle.com/datasets/new-york-city/nyc-property-sales)


# Loading Dataset¶

In [ ]:
import pandas as pd

# Muat dataset ke dalam DataFrame
data = pd.read_csv('./Dataset/nyc-rolling-sales.csv')

# Tampilkan informasi dasar dari dataset
data_info = data.info()

# Tampilkan statistik deskriptif dari dataset
data_description = data.describe()

data_info, data_description


# Informasi Dasar dari Dataset:

    Dataset ini terdiri dari 84,548 entri.
    Ada 22 kolom dalam dataset.
    Tidak ada nilai yang hilang (missing values) pada setiap kolom.
    Terdapat beberapa kolom dengan tipe data objek yang mungkin perlu dikonversi, seperti kolom 'SALE PRICE', 'LAND SQUARE FEET', dan 'GROSS SQUARE FEET' yang seharusnya berisi nilai numerik.

Statistik Deskriptif:

    Kolom 'BOROUGH' memiliki nilai dari 1 hingga 5, yang mewakili daerah di New York City.
    Kolom 'YEAR BUILT' memiliki beberapa entri dengan nilai 0, yang mungkin perlu diperbaiki atau dihapus.
    Kolom 'SALE PRICE' saat ini berupa tipe data objek, jadi kita perlu mengubahnya menjadi tipe data numerik untuk analisis lebih lanjut.

Selanjutnya, kita akan melanjutkan dengan pra-pemrosesan data.

In [ ]:
# Konversi kolom 'SALE PRICE', 'LAND SQUARE FEET', dan 'GROSS SQUARE FEET' ke tipe data numerik
data['SALE PRICE'] = pd.to_numeric(data['SALE PRICE'], errors='coerce')
data['LAND SQUARE FEET'] = pd.to_numeric(data['LAND SQUARE FEET'], errors='coerce')
data['GROSS SQUARE FEET'] = pd.to_numeric(data['GROSS SQUARE FEET'], errors='coerce')

# Filter out transaksi dengan 'SALE PRICE' sebesar $0
data = data[data['SALE PRICE'] > 0]

# Filter out data dengan 'YEAR BUILT' sebesar 0
data = data[data['YEAR BUILT'] > 0]

# Tampilkan informasi dataset setelah pra-pemrosesan
data_info_cleaned = data.info()
data_description_cleaned = data.describe()

data_info_cleaned, data_description_cleaned


# Informasi Dataset Setelah Pra-pemrosesan:

    * Setelah menghilangkan transaksi dengan harga jual $0 dan bangunan dengan 'YEAR BUILT' sebesar 0, kita memiliki 55,449 entri.
    * Kolom 'SALE PRICE', 'LAND SQUARE FEET', dan 'GROSS SQUARE FEET' telah dikonversi menjadi tipe data numerik.
    * Ada nilai yang hilang di kolom 'LAND SQUARE FEET' dan 'GROSS SQUARE FEET', yang mungkin perlu ditangani nantinya tergantung analisis yang akan dilakukan.

Statistik Deskriptif Setelah Pra-pemrosesan:

    * Rata-rata harga jual properti adalah sekitar $1,471,886 dengan deviasi standar sebesar $12,573,380.
    * Nilai minimum 'YEAR BUILT' adalah 1111, yang sepertinya merupakan kesalahan input dan mungkin perlu diperbaiki.
    * Nilai maksimum dari 'SALE PRICE' adalah $2,210,000,000, menunjukkan bahwa ada beberapa properti yang dijual dengan harga yang sangat tinggi.

Selanjutnya, kita akan melanjutkan dengan analisis statistik sesuai dengan rencana yang telah dibuat sebelumnya. Pertama, mari kita analisis Ukuran Tendensi Sentra

# Menghitung Ukuran Tendensi Sentral: ¶

In [ ]:
# Menghitung Ukuran Tendensi Sentral

# Mean
mean_sale_price = data['SALE PRICE'].mean()
mean_land_square_feet = data['LAND SQUARE FEET'].mean()
mean_gross_square_feet = data['GROSS SQUARE FEET'].mean()

# Median
median_sale_price = data['SALE PRICE'].median()
median_land_square_feet = data['LAND SQUARE FEET'].median()
median_gross_square_feet = data['GROSS SQUARE FEET'].median()

# Modus
mode_sale_price = data['SALE PRICE'].mode()[0]  # Mengambil nilai pertama jika ada beberapa mode
mode_land_square_feet = data['LAND SQUARE FEET'].mode()[0]
mode_gross_square_feet = data['GROSS SQUARE FEET'].mode()[0]

mean_sale_price, median_sale_price, mode_sale_price, mean_land_square_feet, median_land_square_feet, mode_land_square_feet, mean_gross_square_feet, median_gross_square_feet, mode_gross_square_feet


# Ukuran Tendensi Sentral:

Harga Jual (SALE PRICE):

    * Rata-rata (Mean): $1,471,886
    * Median: $620,000
    * Modus: $10 (Hal ini menarik karena walaupun kita telah memfilter harga $0, masih ada properti yang dijual dengan harga sangat rendah yaitu $10.)

Luas Tanah (LAND SQUARE FEET):

    * Rata-rata (Mean): 3,524.59 sqft
    * Median: 2,300 sqft
    * Modus: 0.0 sqft (Hal ini menunjukkan banyak properti yang memiliki luas tanah yang dicatat sebagai nol.)

Luas Bangunan (GROSS SQUARE FEET):

    * Rata-rata (Mean): 3,753.27 sqft
    * Median: 1,606 sqft
    * Modus: 0.0 sqft (Mirip dengan 'LAND SQUARE FEET', banyak properti yang memiliki luas bangunan yang dicatat sebagai nol.)

Langkah berikutnya adalah menghitung Ukuran Penyebaran. Mari kita lakukan analisis tersebut.

# Menghitung Ukuran Penyebaran:¶

In [ ]:
# Menghitung Ukuran Penyebaran

# Rentang (Range)
range_sale_price = data['SALE PRICE'].max() - data['SALE PRICE'].min()
range_land_square_feet = data['LAND SQUARE FEET'].max() - data['LAND SQUARE FEET'].min()
range_gross_square_feet = data['GROSS SQUARE FEET'].max() - data['GROSS SQUARE FEET'].min()

# Varians
variance_sale_price = data['SALE PRICE'].var()
variance_land_square_feet = data['LAND SQUARE FEET'].var()
variance_gross_square_feet = data['GROSS SQUARE FEET'].var()

# Deviasi Standar (Standard Deviation)
std_sale_price = data['SALE PRICE'].std()
std_land_square_feet = data['LAND SQUARE FEET'].std()
std_gross_square_feet = data['GROSS SQUARE FEET'].std()

range_sale_price, variance_sale_price, std_sale_price, range_land_square_feet, variance_land_square_feet, std_land_square_feet, range_gross_square_feet, variance_gross_square_feet, std_gross_square_feet


# Ukuran Penyebaran:

Harga Jual (SALE PRICE):

    * Rentang (Range): $2,209,999,999
    * Varians: $158,089,890,250,701.7
    * Deviasi Standar (Standard Deviation): $12,573,380.22

Luas Tanah (LAND SQUARE FEET):

    * Rentang (Range): 4,228,300 sqft
    * Varians: 1,270,195,313.46 sqft^2
    * Deviasi Standar (Standard Deviation): 35,639.80 sqft

Luas Bangunan (GROSS SQUARE FEET):

    * Rentang (Range): 3,750,565 sqft
    * Varians: 949,879,275.15 sqft^2
    * Deviasi Standar (Standard Deviation): 30,820.11 sqft

Kita sekarang dapat melanjutkan dengan analisis Distribusi Data untuk mengetahui bagaimana distribusi data kita. Mari kita visualisasikan distribusi data untuk harga jual, luas tanah, dan luas bangunan menggunakan histogram.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(20, 6))

# Distribusi Harga Jual
plt.subplot(1, 3, 1)
sns.histplot(data['SALE PRICE'], bins=100, color='blue', kde=True)
plt.title('Distribusi Harga Jual')
plt.xlabel('Harga Jual')
plt.ylabel('Frekuensi')
plt.yscale('log')  # Menggunakan skala log untuk mempermudah visualisasi
plt.grid(True, which="both", ls="--", linewidth=0.5)

# Distribusi Luas Tanah
plt.subplot(1, 3, 2)
sns.histplot(data['LAND SQUARE FEET'].dropna(), bins=100, color='green', kde=True)  # Menghilangkan NaN values
plt.title('Distribusi Luas Tanah')
plt.xlabel('Luas Tanah (sqft)')
plt.ylabel('Frekuensi')
plt.yscale('log')
plt.grid(True, which="both", ls="--", linewidth=0.5)

# Distribusi Luas Bangunan
plt.subplot(1, 3, 3)
sns.histplot(data['GROSS SQUARE FEET'].dropna(), bins=100, color='red', kde=True)  # Menghilangkan NaN values
plt.title('Distribusi Luas Bangunan')
plt.xlabel('Luas Bangunan (sqft)')
plt.ylabel('Frekuensi')
plt.yscale('log')
plt.grid(True, which="both", ls="--", linewidth=0.5)

plt.tight_layout()
plt.show()


# Distribusi Data:

Harga Jual (SALE PRICE):

    Sebagian besar properti dijual dengan harga di bawah $2,000,000, tetapi ada beberapa properti dengan harga jual yang sangat tinggi, hingga $2,210,000

# Interval Keyakinan

Interval keyakinan memberikan perkiraan tentang rentang di mana rata-rata populasi mungkin berada, dengan tingkat keyakinan tertentu.

In [ ]:
import scipy.stats as stats

confidence_level = 0.95
degrees_freedom = data['SALE PRICE'].count() - 1
mean_sale_price = data['SALE PRICE'].mean()
std_error = data['SALE PRICE'].std() / (data['SALE PRICE'].count()**0.5)

confidence_interval = stats.t.interval(confidence_level, degrees_freedom, mean_sale_price, std_error)
print("95% Confidence Interval for Sale Price:", confidence_interval)


Dengan interval keyakinan 95%, kita dapat mengatakan bahwa kita memiliki keyakinan 95% bahwa rata-rata harga jual properti di New York City sebenarnya berada dalam kisaran \$1,367,230 hingga \$1,576,541.

# Pengujian Hipotesis

Dalam bagian ini, kita akan memeriksa apakah ada perbedaan harga antara properti yang dibangun pada tahun 1900-2000 dengan yang dibangun setelah tahun 2001.

In [ ]:
group_1900_2000 = data[(data['YEAR BUILT'] >= 1900) & (data['YEAR BUILT'] <= 2000)]['SALE PRICE']
group_after_2001 = data[data['YEAR BUILT'] > 2001]['SALE PRICE']

t_stat, p_value = stats.ttest_ind(group_1900_2000, group_after_2001, equal_var=False)

print("T-statistic:", t_stat)
print("P-value:", p_value)


T-statistik: Nilai -6.788 menunjukkan bahwa rata-rata sampel kita jauh lebih rendah dari rata-rata populasi yang diharapkan.
    P-value: Dengan P-value mendekati 0, kita memiliki bukti kuat untuk menolak hipotesis nol, menunjukkan adanya perbedaan yang signifikan secara statistik.

